## import library

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
# built-in
import re, codecs
import time
import os

# pandas
import pandas as pd
import numpy as np

# nlp
import nltk

# sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# plottong
import matplotlib.pyplot as plt


In [3]:
import sklearn
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold,train_test_split,cross_val_score,cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report,accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import seaborn as sns
import matplotlib.pyplot as plt

import pickle

In [5]:

import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

from nltk.corpus import stopwords

In [6]:
import random

def set_seeds(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    # tf.random.set_random_seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)

set_seeds(seed=42)

## read data

In [7]:
dataset_name_type = 'enpt' # en; all; enpt

In [8]:
# import pandas as pd

# def read_semeval_data(data_type='train', language='EN'):
#     base_path = '/content/drive/MyDrive/i2nlp_project/'
#     if data_type == 'train':
#         annotation = f'{base_path}/training_data/{language}/subtask-2-annotations.txt'
#         directory_path = f'{base_path}/training_data/{language}/raw-documents'
#     elif data_type == 'test':
#         annotation = f'{base_path}/dev_data/{language}/subtask-2-annotations.txt'
#         directory_path = f'{base_path}/dev_data/{language}/subtask-2-documents'


#     label_info = pd.read_csv(annotation, sep='\t', header=None)
#     label_info.columns = ['filename', 'narrative', 'sub_narrative']

#     file_info = []
#     for filename in os.listdir(directory_path):
#         file_path = os.path.join(directory_path, filename)
#         if os.path.isfile(file_path):
#             with open(file_path, "r", encoding="utf-8") as file:
#                 file_info.append((filename, file.read()))

#     text_df = pd.DataFrame(file_info, columns=['filename', 'text'])
#     df = label_info.merge(text_df, on='filename', how='inner')
#     df['language'] = language
#     # df['narrative_labels'] = df['narrative'].apply(lambda x: x.split(';'))
#     # df['sub_narrative_labels'] = df['sub_narrative'].apply(lambda x: x.split(';'))
#     return df

# train_df = pd.concat([
#     read_semeval_data(data_type='train', language='EN'),
#     # read_semeval_data(data_type='train', language='PT'),
#     # read_semeval_data(data_type='train', language='BG'),
#     # read_semeval_data(data_type='train', language='HI'),
#     # read_semeval_data(data_type='train', language='RU')
# ], ignore_index=True)
# test_df = pd.concat([
#     read_semeval_data(data_type='test', language='EN'),
#     # read_semeval_data(data_type='test', language='PT'),
#     # read_semeval_data(data_type='test', language='BG'),
#     # read_semeval_data(data_type='test', language='HI'),
#     # read_semeval_data(data_type='test', language='RU')
# ], ignore_index=True)

# train_df.to_csv(f'/content/drive/MyDrive/i2nlp_project/dataset/train_{dataset_name_type}.csv', index=False)
# test_df.to_csv(f'/content/drive/MyDrive/i2nlp_project/dataset/test_{dataset_name_type}.csv', index=False)

In [9]:
train_df = pd.read_csv(f'/content/drive/MyDrive/i2nlp_project/dataset/train_{dataset_name_type}.csv')
test_df = pd.read_csv(f'/content/drive/MyDrive/i2nlp_project/dataset/test_{dataset_name_type}.csv')


train_df['narrative_labels'] = train_df['narrative'].apply(lambda x: x.split(';'))
test_df['narrative_labels'] = test_df['narrative'].apply(lambda x: x.split(';'))

train_df['sub_narrative_labels'] = train_df['sub_narrative'].apply(lambda x: x.split(';'))
test_df['sub_narrative_labels'] = test_df['sub_narrative'].apply(lambda x: x.split(';'))

# train_df = train_df[train_df['language'] == 'EN']
# test_df = test_df[test_df['language'] == 'EN']

# print(train_df.groupby(['language', 'dataset_type', 'dataset_type_label'])[['filename']].agg('count').rename(columns={'filename': 'count'}))
# print(test_df.groupby(['language', 'dataset_type', 'dataset_type_label'])[['filename']].agg('count').rename(columns={'filename': 'count'}))

# train_df = train_df[(train_df.dataset_type == 'URW') & (train_df.language == 'EN')]
# test_df= test_df[(test_df.dataset_type == 'URW') & (test_df.language == 'EN')]

print(train_df.shape, test_df.shape)

(799, 8) (76, 8)


# Classification start

In [10]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [11]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
from datasets import Dataset

import re

In [12]:
# # Initialize tokenizer
# tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# # Tokenize and chunk data
# def tokenize_and_chunk(documents, labels, chunk_size=512):
#     inputs, targets = [], []
#     for doc, label in zip(documents, labels):
#         tokenized = tokenizer(doc, truncation=False, add_special_tokens=False)
#         input_ids = tokenized["input_ids"]
#         chunks = [input_ids[i:i + chunk_size] for i in range(0, len(input_ids), chunk_size)]

#         # Pad chunks to chunk_size
#         for chunk in chunks:
#             chunk += [tokenizer.pad_token_id] * (chunk_size - len(chunk))

#         inputs.extend(chunks)
#         targets.extend([label] * len(chunks))
#     return inputs, targets


# #-----------------TRAIN-----------------------
# documents = train_df['text']
# labels = train_df['narrative_labels']
# input_chunks, target_chunks = tokenize_and_chunk(documents, labels)
# # Convert labels to binary matrix
# mlb = MultiLabelBinarizer()
# binarized_labels = mlb.fit_transform(target_chunks)

# # Convert input_chunks and binarized_labels to a Hugging Face Dataset
# train_data = {"input_ids": input_chunks, "labels": binarized_labels.astype(float).tolist()}
# train_dataset = Dataset.from_dict(train_data)



# #-----------------TEST------------------------
# # Convert test_df to Hugging Face Dataset
# test_texts = test_df['text']
# test_labels = test_df['narrative_labels']

# # Tokenize and chunk the test dataset
# test_chunks, test_targets = tokenize_and_chunk(test_texts, test_labels)
# test_binarized_labels = mlb.transform(test_targets)

# test_data = {"input_ids": test_chunks, "labels": test_binarized_labels.astype(float).tolist()}
# test_dataset = Dataset.from_dict(test_data)

# Multi-label multi-class classification

In [14]:
from transformers import BertTokenizer
from sklearn.preprocessing import MultiLabelBinarizer
from datasets import Dataset
import pandas as pd

# Initialize tokenizer
model_name = "bert-base-multilingual-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

def tokenize_documents(documents, labels, max_length=512, tokenizer=None):
    """
    Tokenizes documents into single sequences of max length `max_length`.
    Adds [CLS] and [SEP] tokens automatically and pads/truncates to `max_length`.

    Args:
       documents (list): List of text documents to tokenize.
       labels (list): List of labels corresponding to the documents.
       max_length (int): Maximum length for tokenized sequences.
       tokenizer (BertTokenizer): Pre-trained tokenizer instance.

    Returns:
       dict: Dictionary with `input_ids`, `attention_mask`, and `labels`.
    """
    input_ids_list = []
    attention_mask_list = []

    for doc in documents:
        # Tokenize and truncate/pad the document to `max_length`
        tokens = tokenizer(
            doc.lower(),
            max_length=max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        input_ids_list.append(tokens["input_ids"].squeeze(0).tolist())
        attention_mask_list.append(tokens["attention_mask"].squeeze(0).tolist())

    return {
        "input_ids": input_ids_list,
        "attention_mask": attention_mask_list,
        "labels": labels,
    }

# Ensure train_df and test_df are properly loaded Pandas DataFrames


# ----------------- TRAIN -----------------------
train_texts = train_df["text"].tolist()  # Filter and convert column to list
train_labels = train_df["sub_narrative_labels"].tolist()  # Ensure labels are lists

# MultiLabelBinarizer to binarize the labels
mlb = MultiLabelBinarizer()
train_binarized_labels = mlb.fit_transform(train_labels).astype(float).tolist()

train_data = tokenize_documents(train_texts, train_binarized_labels, max_length=512, tokenizer=tokenizer)
train_dataset = Dataset.from_dict(train_data)

# ----------------- TEST ------------------------
test_texts = test_df["text"].tolist()  # Filter and convert column to list
test_labels = test_df["sub_narrative_labels"].tolist()  # Ensure labels are lists

test_binarized_labels = mlb.transform(test_labels).astype(float).tolist()  # Use same MultiLabelBinarizer

test_data = tokenize_documents(test_texts, test_binarized_labels, max_length=512, tokenizer=tokenizer)
test_dataset = Dataset.from_dict(test_data)

# Example output to confirm functionality
print("Number of training samples:", len(train_dataset))
print("Number of test samples:", len(test_dataset))
print("Number of classes:", mlb.classes_.shape)

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['CC: Green policies are geopolitical instruments: Other'] will be ignored
  warnings.warn(


Number of training samples: 799
Number of test samples: 76
Number of classes: (92,)


In [ ]:
import torch
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from transformers import BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, hamming_loss


# Modify loss function to include class weights
class WeightedBCEWithLogitsLoss(torch.nn.Module):
    def __init__(self, weights):
        super().__init__()
        self.weights = weights

    def forward(self, logits, labels):
        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            logits, labels, pos_weight=self.weights
        )
        return loss


# Compute class weights
flat_labels = np.concatenate(train_binarized_labels)
class_weights = compute_class_weight("balanced", classes=np.unique(flat_labels), y=flat_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained(
    model_name, num_labels=len(mlb.classes_)
)
model.config.problem_type = "multi_label_classification"
model.criterion = WeightedBCEWithLogitsLoss(class_weights_tensor)


# Metrics function
def compute_metrics(pred):
    logits, labels = pred
    predictions = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    labels = labels.astype(int)

    # Hamming loss
    hamming = hamming_loss(labels, predictions)

    # Precision, recall, and F1 (micro and macro averages)
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        labels, predictions, average="micro", zero_division=0
    )
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        labels, predictions, average="macro", zero_division=0
    )

    # Subset accuracy
    subset_acc = accuracy_score(labels, predictions)

    # Classification report (as a string)
    class_report = classification_report(labels, predictions, zero_division=0, target_names=mlb.classes_)

    # Print the classification report
    # print("\nClassification Report:")
    # print(class_report)

    return {
        "hamming_loss": hamming,
        "micro_precision": precision_micro,
        "micro_recall": recall_micro,
        "micro_f1": f1_micro,
        "macro_precision": precision_macro,
        "macro_recall": recall_macro,
        "macro_f1": f1_macro,
        "subset_accuracy": subset_acc,
    }


# Training arguments
EPOCHS = 50
model_train_name = 'bert_classifier'
learning_rate = 2e-5
batch_size = 8

# Define training arguments

training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/i2nlp_project/results/{model_train_name}",
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    num_train_epochs=EPOCHS,
    lr_scheduler_type="linear",
    save_strategy="no",  # Prevent saving every model
    report_to="none",
    logging_dir=f"/content/drive/MyDrive/i2nlp_project/logs/{model_train_name}",
    logging_steps=10,
)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

# Update Trainer with the validation dataset and metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train and evaluate
trainer.train()

# Evaluate on the test set
results = trainer.evaluate(eval_dataset=test_dataset)
print("Evaluation results:", results)


In [21]:
model.save_pretrained(f'/content/drive/MyDrive/i2nlp_project/models/bert_classifier_last/')
tokenizer.save_pretrained(f'/content/drive/MyDrive/i2nlp_project/models/bert_classifier_last/')

In [17]:
def compute_metrics(pred):
    logits, labels = pred
    predictions = (torch.sigmoid(torch.tensor(logits)) > 0.5).int().numpy()
    labels = labels.astype(int)

    # Hamming loss
    hamming = hamming_loss(labels, predictions)

    # Precision, recall, and F1 (micro and macro averages)
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        labels, predictions, average="micro", zero_division=0
    )
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        labels, predictions, average="macro", zero_division=0
    )

    # Subset accuracy
    subset_acc = accuracy_score(labels, predictions)

    # Classification report (as a string)
    class_report = classification_report(labels, predictions, zero_division=0, target_names=mlb.classes_)

    # Print the classification report
    # print("\nClassification Report:")
    # print(class_report)

    return {
        "hamming_loss": hamming,
        "micro_precision": precision_micro,
        "micro_recall": recall_micro,
        "micro_f1": f1_micro,
        "macro_precision": precision_macro,
        "macro_recall": recall_macro,
        "macro_f1": f1_macro,
        "subset_accuracy": subset_acc,
    }
# Update Trainer with the validation dataset and metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train and evaluate
trainer.train()

# Evaluate on the test set
results = trainer.evaluate(eval_dataset=test_dataset)
print("Evaluation results:", results)

Epoch,Training Loss,Validation Loss,Hamming Loss,Micro Precision,Micro Recall,Micro F1,Macro Precision,Macro Recall,Macro F1,Subset Accuracy,Runtime,Samples Per Second,Steps Per Second
1,0.046400,0.086194,0.025887,0.612245,0.156250,0.248963,0.073887,0.039685,0.047817,0.144737,1.105400,68.756000,9.047000
2,0.053100,0.079978,0.025029,0.654545,0.187500,0.291498,0.089883,0.055135,0.061991,0.131579,1.154500,65.828000,8.662000
3,0.049600,0.082012,0.025601,0.603175,0.197917,0.298039,0.095411,0.072876,0.075037,0.144737,1.120000,67.855000,8.928000
4,0.049300,0.081095,0.024886,0.666667,0.187500,0.292683,0.103649,0.055392,0.066023,0.157895,1.141700,66.566000,8.759000
5,0.049500,0.081854,0.025887,0.617021,0.151042,0.242678,0.063225,0.032115,0.039354,0.144737,1.136600,66.864000,8.798000
6,0.039200,0.080720,0.025458,0.625000,0.182292,0.282258,0.101329,0.060667,0.069014,0.157895,1.132900,67.087000,8.827000
7,0.037300,0.081030,0.025458,0.640000,0.166667,0.264463,0.103470,0.045689,0.058362,0.144737,1.138200,66.770000,8.785000
8,0.035100,0.081267,0.025744,0.607143,0.177083,0.274194,0.081896,0.048717,0.055680,0.144737,1.137700,66.803000,8.790000
9,0.039500,0.079276,0.025458,0.625000,0.182292,0.282258,0.107583,0.061055,0.073301,0.131579,1.132400,67.117000,8.831000
10,0.032200,0.082274,0.026030,0.575758,0.197917,0.294574,0.091731,0.056437,0.064890,0.171053,1.137100,66.839000,8.795000


Evaluation results: {'eval_loss': 0.09096293896436691, 'eval_hamming_loss': 0.02688787185354691, 'eval_micro_precision': 0.525, 'eval_micro_recall': 0.21875, 'eval_micro_f1': 0.3088235294117647, 'eval_macro_precision': 0.10606452726017944, 'eval_macro_recall': 0.0759879160694378, 'eval_macro_f1': 0.08540973840475792, 'eval_subset_accuracy': 0.15789473684210525, 'eval_runtime': 1.122, 'eval_samples_per_second': 67.735, 'eval_steps_per_second': 8.913, 'epoch': 50.0}


In [26]:
trainer.evaluate(eval_dataset=train_dataset)


Classification Report:
                                                                                                                   precision    recall  f1-score   support

                                        CC: Amplifying Climate Fears: Amplifying existing fears of global warming       1.00      1.00      1.00        35
                                                      CC: Amplifying Climate Fears: Doomsday scenarios for humans       1.00      1.00      1.00        34
                                                                              CC: Amplifying Climate Fears: Other       1.00      1.00      1.00        34
                                                              CC: Climate change is beneficial: CO2 is beneficial       1.00      1.00      1.00        46
                                             CC: Climate change is beneficial: Temperature increase is beneficial       1.00      1.00      1.00        34
                                             

{'eval_loss': 0.007371334359049797,
 'eval_hamming_loss': 1.3316288484073719e-05,
 'eval_micro_precision': 0.9996744261761354,
 'eval_micro_recall': 1.0,
 'eval_micro_f1': 0.9998371865841745,
 'eval_macro_precision': 0.9996004407769112,
 'eval_macro_recall': 1.0,
 'eval_macro_f1': 0.9997978890635527,
 'eval_subset_accuracy': 0.9994407158836689,
 'eval_runtime': 25.7594,
 'eval_samples_per_second': 69.411,
 'eval_steps_per_second': 8.696,
 'epoch': 50.0}

In [27]:
trainer.save_model(f'/content/drive/MyDrive/i2nlp_project/models/bert_512_augmented_en/')

In [15]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Load the model and tokenizer from the saved path
model_path = '/content/drive/MyDrive/i2nlp_project/models/bert_classifier_last/'

model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer = BertTokenizer.from_pretrained(model_path)

# Ensure the model is in evaluation mode
model.eval()



BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [24]:
# Create DataLoader for test set (optional, for batch processing)
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support, hamming_loss

def collate_fn(batch):
    input_ids = torch.tensor([item['input_ids'] for item in batch])
    attention_mask = torch.tensor([item['attention_mask'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch], dtype=torch.float)  # Use float for multi-label
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

# Create DataLoader for the test set
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=collate_fn)

# Collect predictions and labels for evaluation
all_logits = []
all_labels = []

# Loop through test set and get predictions
for batch in test_loader:
    inputs = {
        "input_ids": batch["input_ids"],
        "attention_mask": batch["attention_mask"]
    }
    labels = batch["labels"]

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        all_logits.append(logits)
        all_labels.append(labels)

# Convert logits and labels into tensors and concatenate
all_logits = torch.cat(all_logits, dim=0)
all_labels = torch.cat(all_labels, dim=0)


In [30]:
def compute_metrics(pred):
    logits, labels = pred
    predictions = (torch.sigmoid(torch.tensor(logits)) > 0.3).int().numpy()
    labels = labels.astype(int)

    # Hamming loss
    hamming = hamming_loss(labels, predictions)

    # Precision, recall, and F1 (micro and macro averages)
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        labels, predictions, average="micro", zero_division=0
    )
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        labels, predictions, average="macro", zero_division=0
    )

    # Subset accuracy
    subset_acc = accuracy_score(labels, predictions)

    # Classification report (as a string)
    class_report = classification_report(labels, predictions, zero_division=0, target_names=mlb.classes_)

    # Print the classification report
    # print("\nClassification Report:")
    # print(class_report)

    return {
        "hamming_loss": hamming,
        "micro_precision": precision_micro,
        "micro_recall": recall_micro,
        "micro_f1": f1_micro,
        "macro_precision": precision_macro,
        "macro_recall": recall_macro,
        "macro_f1": f1_macro,
        "subset_accuracy": subset_acc,
    }

# Compute the metrics using the logits and labels
metrics = compute_metrics((all_logits.numpy(), all_labels.numpy()))

# Print the results
print("Computed Metrics:")
for key, value in metrics.items():
    print(f"{key}: {value}")

Computed Metrics:
hamming_loss: 0.02917620137299771
micro_precision: 0.45384615384615384
micro_recall: 0.3072916666666667
micro_f1: 0.36645962732919257
macro_precision: 0.14227732126759585
macro_recall: 0.14807941153049847
macro_f1: 0.13162103334598607
subset_accuracy: 0.15789473684210525


In [48]:
predictions = (torch.sigmoid(all_logits) > 0.3).int().numpy()
result_df_save = test_df[['filename', 'language']] # , 'narrative', 'sub_narrative'
# result_df_save['predicted_labels'] = mlb.inverse_transform(predictions)
result_df_save['narrative'] = [';'.join(list(set(':'.join(j.split(':')[:2]) for j in i))) if i else 'Other' for i in mlb.inverse_transform(predictions)]
result_df_save['sub_narrative'] = [';'.join(i if i else ['Other']) for i in mlb.inverse_transform(predictions)]
# result_df_save.to_csv('/content/drive/MyDrive/i2nlp_project/predicted/bert_en_25.csv', index=False, sep='\t', header=None)
result_df_save[result_df_save['language'] == 'EN'][['filename', 'narrative', 'sub_narrative']]\
.to_csv('/content/drive/MyDrive/i2nlp_project/predicted/dev_bert_en_25.txt', index=False, sep='\t', header=None)
result_df_save[result_df_save['language'] == 'PT'][['filename', 'narrative', 'sub_narrative']]\
.to_csv('/content/drive/MyDrive/i2nlp_project/predicted/dev_bert_pt_25.txt', index=False, sep='\t', header=None)
result_df_save

<ipython-input-48-6b5aa35e0221>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df_save['narrative'] = [';'.join(list(set(':'.join(j.split(':')[:2]) for j in i))) if i else 'Other' for i in mlb.inverse_transform(predictions)]
<ipython-input-48-6b5aa35e0221>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df_save['sub_narrative'] = [';'.join(i if i else ['Other']) for i in mlb.inverse_transform(predictions)]


,filename,language,narrative,sub_narrative
0,EN_UA_DEV_100012.txt,EN,Other,Other
1,EN_CC_200053.txt,EN,CC: Criticism of institutions and authorities,CC: Criticism of institutions and authorities:...
2,EN_CC_200040.txt,EN,CC: Criticism of climate movement;CC: Criticis...,CC: Criticism of climate movement: Climate mov...
3,EN_CC_200070.txt,EN,Other,Other
4,EN_UA_DEV_100034.txt,EN,"URW: Discrediting the West, Diplomacy;URW: Dis...",URW: Discrediting Ukraine: Discrediting Ukrain...
...,...,...,...,...
71,PT_207.txt,PT,CC: Criticism of climate policies,CC: Criticism of climate policies: Other
72,PT_217.txt,PT,CC: Amplifying Climate Fears,CC: Amplifying Climate Fears: Amplifying exist...
73,PT_204.txt,PT,CC: Criticism of institutions and authorities,CC: Criticism of institutions and authorities:...
74,PT_229.txt,PT,CC: Amplifying Climate Fears,CC: Amplifying Climate Fears: Other


# OLD CODE

In [ ]:
model_name = "bert-base-multilingual-cased"

# Initialize tokenizer (example: multilingual cased)
tokenizer = BertTokenizer.from_pretrained(model_name)

def tokenize_and_chunk(
    documents,
    labels,
    chunk_size=512,
    tokenizer=tokenizer
):
    """
    Tokenizes and chunks documents into segments of max length `chunk_size`.
    Each chunk is prepended with [CLS] and appended with [SEP] tokens.

    Returns:
       input_ids_list: 2D list of token IDs (one sublist per chunk).
       attention_mask_list: 2D list (same shape as input_ids_list).
       labels_list: 1D list of labels aligned with the chunks.
    """

    input_ids_list = []
    attention_mask_list = []
    labels_list = []

    # Subtract 2 to reserve space for [CLS] and [SEP]
    effective_chunk_size = chunk_size - 2

    for doc, label in zip(documents, labels):
        # 1) Tokenize without adding special tokens or truncation
        tokens = tokenizer(doc, add_special_tokens=False, truncation=False)
        all_ids = tokens["input_ids"]

        # 2) Split the `all_ids` into chunks of `effective_chunk_size`
        for i in range(0, len(all_ids), effective_chunk_size):
            chunk_ids = all_ids[i : i + effective_chunk_size]

            # Insert [CLS] and [SEP]
            chunk_ids = [tokenizer.cls_token_id] + chunk_ids + [tokenizer.sep_token_id]

            # Create attention mask (1 for real tokens, 0 for padding)
            attention_mask = [1] * len(chunk_ids)

            # 3) Pad up to `chunk_size` if needed
            if len(chunk_ids) < chunk_size:
                pad_length = chunk_size - len(chunk_ids)
                chunk_ids += [tokenizer.pad_token_id] * pad_length
                attention_mask += [0] * pad_length

            # Collect results
            input_ids_list.append(chunk_ids)
            attention_mask_list.append(attention_mask)
            labels_list.append(label)

    return input_ids_list, attention_mask_list, labels_list


# ----------------- TRAIN -----------------------
documents = train_df["text"]
labels = train_df["dataset_type"]

input_chunks, attn_masks, target_chunks = tokenize_and_chunk(documents, labels, chunk_size=512)

# Convert labels (e.g., multi-label) to binary matrix if needed
mlb = MultiLabelBinarizer()
binarized_labels = mlb.fit_transform(target_chunks)

# Convert to a Hugging Face Dataset
train_data = {
    "input_ids": input_chunks,
    "attention_mask": attn_masks,
    "labels": binarized_labels.astype(float).tolist()
}
train_dataset = Dataset.from_dict(train_data)

# ----------------- TEST ------------------------
test_texts = test_df["text"]
test_labels = test_df["dataset_type"]

test_chunks, test_attn_masks, test_targets = tokenize_and_chunk(test_texts, test_labels, chunk_size=512)
test_binarized_labels = mlb.transform(test_targets)

test_data = {
    "input_ids": test_chunks,
    "attention_mask": test_attn_masks,
    "labels": test_binarized_labels.astype(float).tolist()
}
test_dataset = Dataset.from_dict(test_data)


In [ ]:
# Modify loss function to include class weights
class WeightedBCEWithLogitsLoss(torch.nn.Module):
    def __init__(self, weights):
        super().__init__()
        self.weights = weights

    def forward(self, logits, labels):
        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            logits, labels, pos_weight=self.weights
        )
        return loss


# Compute class weights
flat_labels = np.concatenate(binarized_labels)
class_weights = compute_class_weight("balanced", classes=np.unique(flat_labels), y=flat_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

# Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained(
    model_name, num_labels=len(mlb.classes_)
)
model.config.problem_type = "multi_label_classification"
model.criterion = WeightedBCEWithLogitsLoss(class_weights_tensor)

In [ ]:
from sklearn.metrics import hamming_loss, precision_recall_fscore_support, accuracy_score, classification_report
import numpy as np
from transformers import DataCollatorWithPadding


EPOCHS = 10
model_train_name = 'bert512_2025_01'

# Define training arguments
training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/i2nlp_project/results/{model_train_name}",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    num_train_epochs=EPOCHS,
    lr_scheduler_type="linear",
    save_strategy="no",  # Prevent saving every model
    report_to="none",
    logging_dir=f"/content/drive/MyDrive/i2nlp_project/logs/{model_train_name}",
    logging_steps=10,
)


# Metrics function

def compute_metrics(pred):
    logits, labels = pred
    predictions = (torch.sigmoid(torch.tensor(logits)) > 0.2).int().numpy()
    labels = labels.astype(int)

    print("Logits:\n", logits[:3])
    print("Sigmoid:\n", torch.sigmoid(torch.tensor(logits[:3])))
    print("Predictions:\n", predictions[:3])
    print("Labels:\n", labels[:3])

    # Hamming loss
    hamming = hamming_loss(labels, predictions)

    # Precision, recall, and F1 (micro and macro averages)
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        labels, predictions, average="micro", zero_division=0
    )
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        labels, predictions, average="macro", zero_division=0
    )

    # Subset accuracy
    subset_acc = accuracy_score(labels, predictions)

    # Classification report (as a string)
    class_report = classification_report(labels, predictions, zero_division=0, target_names=mlb.classes_)

    # Print the classification report
    print("\nClassification Report:")
    print(class_report)

    return {
        "hamming_loss": hamming,
        "micro_precision": precision_micro,
        "micro_recall": recall_micro,
        "micro_f1": f1_micro,
        "macro_precision": precision_macro,
        "macro_recall": recall_macro,
        "macro_f1": f1_macro,
        "subset_accuracy": subset_acc,
    }


data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

In [ ]:

# Update Trainer with validation dataset and metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


# Train and evaluate
trainer.train()

# Evaluate on the test set
results = trainer.evaluate(eval_dataset=test_dataset)
print("Evaluation results:", results)

In [ ]:
trainer.save_model(f'/content/drive/MyDrive/i2nlp_project/models/{model_train_name}/')

In [ ]:

# Evaluate on the test set
results = trainer.evaluate(eval_dataset=train_dataset)
results